In [2]:
import splat
import wisps
import matplotlib.pyplot as plt
from wisps.data_analysis import selection_criteria as sel_crt

import numpy as np
import pandas as pd

import numba
import matplotlib as mpl
mpl.rcParams['font.size'] = 18

from itertools import combinations
#%%capture  output
import itertools
from tqdm import tqdm


import seaborn as sns 
cmap=sns.light_palette((260, 75, 60), input="husl", as_cmap=True)
from tensorflow import keras

from scipy import stats

In [3]:
ucds=pd.read_pickle('~/research/wisps/libraries/real_ucds.pkl')
#selection objects 
rfdict=pd.read_pickle(wisps.OUTPUT_FILES+'/random_forest_classifier.pkl') #the classifier 
indices_to_use= pd.read_pickle(wisps.OUTPUT_FILES+'/best_indices_to_use.pkl')
neural_network= keras.models.load_model(wisps.OUTPUT_FILES+'/deep_model_september23.h5')
#neural_network= keras.models.load_model(wisps.OUTPUT_FILES+'/conv_model_withspt_snr_september17.h5')

In [4]:
import wisps.data_analysis.selection_criteria as sel_crt
indices_to_use= pd.read_pickle(wisps.OUTPUT_FILES+'/best_indices_to_use.pkl')
slc_crts=sel_crt.crts_from_file()

In [5]:
big=wisps.get_big_file()

In [6]:
#define a number of selectors 
#each selection should a column of zeros and ones corresponding
#to where objects were selected
#each selector input is the simulated df
def f_test_fx(x, df1, df2):
    return stats.f.cdf(x, df1, df2)

def select_by_indices(df, idx, spt_range):
    print(spt_range)
    bs=idx.shapes
    bx=[x for x in bs if x.shape_name==spt_range][0]
    _, bools= bx._select(np.array([df[idx.xkey].values, df[idx.ykey].values]))
    return bools
        

def apply_scale(x):
    ##remove nans 
    ##this is the same scaling used 
    ## same scaling used for 
    #replace nans
    y=x
    if np.isnan(y) or np.isinf(y) or abs(y) > 1e10:
        y=-99
    return y


def select_by_random_forest(df):
    #use the classification given by my rf classifier
    
    rf=rfdict['classifier']
    #min_max_scaler=rfdict['sclr']
    features=rfdict['feats']
    
    #apply logs to problematic features the same way I did on my classification
    pred_df=df.copy()
    
    for c in features:
        if c not in ['spt', 'f_test', 'x']:
            pred_df.assign(c=np.log10(pred_df[c].apply(apply_scale)))
        else:
            pred_df.assign(c=pred_df[c].apply(apply_scale))
            
    pred_df[features]=pred_df[features].applymap(apply_scale)
    #make predictions
    probs=rf.predict_proba(pred_df[features].values)
    
    #use my selection probability cutoffs
    #return the predictions
    
    labels=np.logical_or.reduce((probs[:, 2] > .8, probs[:,3] >.8 , probs[:,4] >0.8))
 
    return labels



def select_by_neuralnet(df):
    #define features (start with indices alone)
    #apply logs to problematic features the same way I did on my classification
    features=rfdict['feats']
    
    pred_df=df.copy()
    
    for c in features:
        if c not in ['spt']:
            pred_df.assign(c=np.log10(pred_df[c].apply(apply_scale)))
        else:
            pred_df.assign(c=pred_df[c].apply(apply_scale))
            
    pred_df[features]=pred_df[features].applymap(apply_scale)
    #probs= neural_network.predict(  pred_df[features].values)
    #need to reshape
    #probs=neural_network.predict(  pred_df[features].values.reshape(-1, len(features), 1))
    
    #my cuts
    #labels=np.logical_or.reduce((probs[:, 2] > .7, probs[:,3] >.5 , probs[:,4] >0.5))
    #labels=probs[:,0] <0.5
    labels=neural_network.predict_classes(  pred_df[features].values.reshape(-1, len(features), 1))
 
    return labels



In [7]:
ucds['x']=ucds.spectra.apply(lambda x: x.spex_chi/x.line_chi)
ucds['line_chi']=ucds.spectra.apply(lambda x: x.line_chi)
ucds['spex_chi']=ucds.spectra.apply(lambda x: x.spex_chi)
ucds['f_test']=ucds.spectra.apply(lambda x: x.f_test)

In [8]:
#ucds.columns

In [9]:
#ucds['x']=

In [10]:
ls=select_by_neuralnet(ucds)

In [11]:
ls

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [12]:
len(ucds[ls>=2])

208

In [13]:
lp=select_by_neuralnet(big)

In [14]:
len(big[np.logical_and(big.snr1>=3, lp>=2)])-129

1761

In [15]:


rfselection=pd.read_csv(wisps.OUTPUT_FILES+'/selected_by_rf.csv')
#nnres=pd.read_pickle(wisps.OUTPUT_FILES+'/deep_neural_net_classifier_results.pkl')

#rfselection.shape

#nnsel= pd.concat(nnres['selected'])

nnselection =pd.DataFrame()

nnselection['grism_id']=ucds.grism_id[ls]
#nnselection=nnselection.drop_duplicates('grism_id')



labelled=wisps.Annotator.group_by_spt(ucds, spt_label='spt', assign_number=False)

for idxk, k in indices_to_use:
    idx=slc_crts[idxk]
    labelled['selected_by_{}'.format(k)]= select_by_indices(labelled, idx, k)

labelled['index_label']=np.logical_or.reduce([labelled['selected_by_{}'.format(x)].values 
                                              for x in  np.vstack(indices_to_use)[:,1]])

#add labels for index selection
labelled['selected_by_rf']=labelled.grism_id.str.lower().isin(rfselection.grism_id.str.lower())
labelled['selected_by_nn']=labelled.grism_id.str.lower().isin(nnselection.grism_id.str.lower())
for g in labelled.groupby('spt_range'):
    df0=g[1]
    print ('{}'.format(g[0]))
    print ('selected by rf {}'.format(len(df0[df0.selected_by_rf])))
    print ('selected by NN {}'.format(len(df0[df0.selected_by_nn])))
    print ('selected by indices {}'.format(len(df0[df0.index_label])))
    print ('selected by rf and indices {}'.format(len(df0[np.logical_and(df0.index_label,df0.selected_by_rf) ])))
    print ('selected by NOT rf and indices {}'.format(len(df0[np.logical_and(df0.index_label,~df0.selected_by_rf) ])))
    print ('selected by rf and NOT indices {}'.format(len(df0[np.logical_and(~df0.index_label,df0.selected_by_rf) ])))
    print ('selected by NN and indices {}'.format(len(df0[np.logical_and(df0.index_label,df0.selected_by_nn) ])))
    print ('selected by NOT NN and indices {}'.format(len(df0[np.logical_and(df0.index_label,~df0.selected_by_nn) ])))
    print ('selected by NN and NOT indices {}'.format(len(df0[np.logical_and(~df0.index_label,df0.selected_by_nn) ])))
    print ('---------------------------------------------------------------- ')

M7-L0
L0-L5
L5-T0
T0-T5
T5-T9
Y dwarfs
subdwarfs
L0-L5
selected by rf 21
selected by NN 0
selected by indices 16
selected by rf and indices 16
selected by NOT rf and indices 0
selected by rf and NOT indices 5
selected by NN and indices 0
selected by NOT NN and indices 16
selected by NN and NOT indices 0
---------------------------------------------------------------- 
L5-T0
selected by rf 9
selected by NN 0
selected by indices 8
selected by rf and indices 8
selected by NOT rf and indices 0
selected by rf and NOT indices 1
selected by NN and indices 0
selected by NOT NN and indices 8
selected by NN and NOT indices 0
---------------------------------------------------------------- 
M7-L0
selected by rf 116
selected by NN 5
selected by indices 175
selected by rf and indices 104
selected by NOT rf and indices 71
selected by rf and NOT indices 12
selected by NN and indices 5
selected by NOT NN and indices 170
selected by NN and NOT indices 0
-------------------------------------------------

In [16]:
len(ucds)

231

In [17]:
import astropy.units as u
(0.5*u.deg**2).to(u.arcmin**2)*(0.1/u.arcmin**2)

<Quantity 180.>

In [18]:
for m in neural_network.metrics:
     print (' {} {}'.format(m.name, m.result()))

 loss 0.0
 cross_entropy 0.0
 precision 0.0
 recall 0.0
 accuracy 0.0
 falseneg 0.0
 falsepos 0.0
 truepos 0.0


### 